In [19]:



import pandas as pd
import numpy as np
import os
from IPython.display import clear_output


pip intall openpyxl

import pandas as pd
import numpy as np
import os
from IPython.display import clear_output

In [20]:
import pandas as pd
import numpy as np
import os
from IPython.display import clear_output

espd068Type = {
    'Repres': 'int64',                 
    'Nome Repres': 'object',          
    'Estab': 'int64',                  
    'Nr Nota Fiscal': 'int64',         
    'Dt Emis Nota': 'str',          
    'Cod Cliente': 'int64',           
    'Nome Cliente': 'object',          
    'Pedido': 'int64',                 
    'Cod Item': 'int64',               
    'Desc Item': 'object',            
    'Qtde Faturada': 'int64',         
    'Valor Unit Item': 'object',       
    'Valor Total Item': 'object',      
    'Qtd Nao Atendida': 'int64',       
    'Valor Nao Atendido': 'object'
}


espd028Type = {
    'Data Implant': 'str',
    'Nr.Pedido': 'int64',
    'Nr.Cliente': 'int64',
    'Nome Abrev': 'object',
    'Nome Repre': 'object',
    'Familia': 'object',
    'Codigo Item': 'int64',
    'Descricao': 'object',
    'Qt.Pedido': 'object',
    'R$ Unit': 'object',
    'Qt.Aloc': 'int64',
    'Vlr.Total': 'object',
    'Est.Disp': 'object',
    'Situação': 'object',
    'Entrega': 'object',
    'MêsAno': 'int64',
    'Lista Presente': 'int64',
    'Observação': 'object'
}

In [21]:
file_path = 'espd0028.txt'
modified_file_path = 'espd0028_modified.txt'

# Leitura do arquivo original
with open(file_path, 'r', encoding='ISO-8859-1') as file:
    lines = file.readlines()

# Modificar o cabeçalho adicionando '@Familia' entre '@Nome Repre' e '@Codigo Item'
header = lines[0].replace('@Nome Repre@', '@Nome Repre@Familia@')
lines[0] = header  # Substitui a primeira linha (cabeçalho)

# Escrever as alterações em um novo arquivo
with open(modified_file_path, 'w', encoding='ISO-8859-1') as modified_file:
    modified_file.writelines(lines)

In [22]:
espd068_path = 'espd0068.txt'
espd028_path = 'espd0028_modified.txt'

espd068 = pd.read_csv(espd068_path, sep='@', quotechar='"', encoding='ISO-8859-1',dtype=espd068Type)
espd028 = pd.read_csv(espd028_path, sep='@', quotechar='"', encoding='ISO-8859-1',dtype=espd028Type)
#espd028.to_excel('espd0028.xlsx', index=False)
espd068['Dt Emis Nota'] = pd.to_datetime(espd068['Dt Emis Nota'], format='%d/%m/%y')
espd028['Data Implant'] = pd.to_datetime(espd028['Data Implant'], format='%d/%m/%y')
espd028 = espd028.dropna(axis=1, how='all')

espd028['Qt.Pedido'] = espd028['Qt.Pedido'].str.replace(',', '.', regex=False)

espd028['Est.Disp'] = espd028['Est.Disp'].str.replace(',', '.', regex=False).astype(float).astype(int)
print(espd028.columns)  # Verifica os nomes das colunas
print(espd028.head())   # Verifica os primeiros registros
print(espd028.dtypes)

Index(['Data Implant', 'Nr.Pedido', 'Nr.Cliente', 'Nome Abrev', 'Nome Repre',
       'Familia', 'Codigo Item', 'Descricao', 'Qt.Pedido', 'R$ Unit',
       'Qt.Aloc', 'Vlr.Total', 'Est.Disp', 'Situação', 'Entrega', 'MêsAno',
       'Lista Presente', 'Observação'],
      dtype='object')
  Data Implant  Nr.Pedido  Nr.Cliente Nome Abrev    Nome Repre   Familia  \
0   2025-05-15    3470072     1582510      FLIP8  DPS COMERCIA  TOYSLEGO   
1   2025-05-15    3470072     1582510      FLIP8  DPS COMERCIA  TOYSLEGO   
2   2025-05-15    3470072     1582510      FLIP8  DPS COMERCIA  TOYSLEGO   
3   2025-05-15    3470072     1582510      FLIP8  DPS COMERCIA  TOYSLEGO   
4   2025-05-15    3470072     1582510      FLIP8  DPS COMERCIA  TOYSLEGO   

   Codigo Item                             Descricao Qt.Pedido   R$ Unit  \
0   4111110369                     Flor de Ameixeira         3  149,6656   
1   4111110698  CAIXA GRANDE DE PEÇAS CRIATIVAS LEGO         1  319,6635   
2   4111110788               

In [23]:
#pedidos_validados =  [3280713, 3281004, 3280910, 3280706, 3280670, 3280049, 3279972, 3279970, 3279830, 3279827, 3279794, 3279684, 3279139, 3279138, 3279112, 3279067, 3278961, 3278849, 3278098, 3278021, 3277955, 3277923, 3277922, 3277911, 3277909, 3276941, 3277572, 3277458, 3277321, 3277320, 3277290, 3277163, 3277141, 3277113, 3277097, 3277091, 3277083, 3277050, 3277030, 3277024, 3277019, 3276995, 3276994, 3276993, 3276965, 3276963, 3276962, 3276960, 3276945, 3276944, 3276346, 3275299, 3274550, 3274448, 3274447, 3274423, 3274401, 3274412, 3274156, 3273706, 3272513, 3268221, 3268162, 3266131, 3266101, 3265952, 3264897, 3264858, 3264825, 3264857, 3263872, 3263811, 3261910, 3261703, 3261909, 3261160, 3259768, 3259340, 3259583, 3258742, 3256359, 3255807, 3255555, 3254971, 3254336, 3254180, 3254066, 3254063, 3252425, 3251764, 3251220, 3251072, 3250768, 3248435, 3248137, 3248110, 3248077, 3247907, 3247451, 3247325, 3246848, 3246733, 3246655, 3245862, 3244223, 3244222, 3243732, 3243723, 3243287, 3243276, 3242329, 3242251, 3241817, 3241708, 3241674, 3241673, 3241669, 3241668, 3241667, 3241666, 3241665, 3241660, 3241659, 3241635, 3241621, 3241619, 3241617, 3241359, 3241317, 3240804, 3240790, 3240769, 3240768, 3240765, 3240733, 3240732, 3240696, 3240694, 3240898, 3240897, 3240297, 3240102, 3240416, 3239741, 3239639, 3239589, 3239580, 3239486, 3239445, 3239407, 3239401, 3239399, 3239374, 3239371, 3239369, 3239367, 3239366, 3239345, 3239301, 3239291, 3239578, 3239124, 3239123, 3239121, 3239090, 3239088, 3239045, 3239010, 3238974, 3238945, 3238944, 3238923, 3238921, 3238884, 3238866, 3230194, 3220965, 3219721, 3219720, 3215194]
#espd028 = espd028[espd028['Nr.Pedido'].isin(pedidos_validados)]

In [24]:
print("Colunas no DataFrame:", espd028.columns.tolist())


Colunas no DataFrame: ['Data Implant', 'Nr.Pedido', 'Nr.Cliente', 'Nome Abrev', 'Nome Repre', 'Familia', 'Codigo Item', 'Descricao', 'Qt.Pedido', 'R$ Unit', 'Qt.Aloc', 'Vlr.Total', 'Est.Disp', 'Situação', 'Entrega', 'MêsAno', 'Lista Presente', 'Observação']


In [25]:
espd028.head(100)

,Data Implant,Nr.Pedido,Nr.Cliente,Nome Abrev,Nome Repre,Familia,Codigo Item,Descricao,Qt.Pedido,R$ Unit,Qt.Aloc,Vlr.Total,Est.Disp,Situação,Entrega,MêsAno,Lista Presente,Observação
0,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111110369,Flor de Ameixeira,3,"149,6656",0,"448,9968",17,Aberto,15/05/25,52025,0,NaN
1,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111110698,CAIXA GRANDE DE PEÇAS CRIATIVAS LEGO,1,"319,6635",0,"319,6635",0,Aberto,15/05/25,52025,0,NaN
2,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111110788,Casa Mágica da Gabby,1,"406,2435",0,"406,2435",6,Aberto,15/05/25,52025,0,NaN
3,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111111023,Base de ConstruçãoVerde,24,"42,6172",0,"1022,8128",0,Aberto,15/05/25,52025,0,NaN
4,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111111025,Base de ConstruçãoAzul,24,"42,6172",0,"1022,8128",21,Aberto,15/05/25,52025,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2025-05-15,3469969,488544,BRINQUEDOLA3,Marcio,TOYSLEGO,4111176426,Casa de Barcos doCastelo de Hogwart,3,"212,1888",3,"636,57",0,Aberto,15/05/25,52025,0,NaN
96,2025-05-15,3469968,1872436,ALEXKIDS,KAUANE,TOYSLEGO,4111110330,McLaren Ayrton Senna,3,"521,51",0,"1564,53",0,Aberto,15/05/25,52025,0,NaN
97,2025-05-15,3469968,1872436,ALEXKIDS,KAUANE,TOYSLEGO,4111110696,CAIXA MEDIA DE PEÇAS CRIATIVAS LEGO,2,"227,27",0,"454,54",1,Aberto,15/05/25,52025,0,NaN
98,2025-05-15,3469968,1872436,ALEXKIDS,KAUANE,TOYSLEGO,4111110698,CAIXA GRANDE DE PEÇAS CRIATIVAS LEGO,1,"389,6",0,"389,6",0,Aberto,15/05/25,52025,0,NaN


In [26]:
# Filtrando o DataFrame para mostrar apenas os itens com 'Codigo Item' igual a 4111160428
itens_codigo_especifico = espd028[espd028['Codigo Item'] == 4111160404]

# Selecionando apenas as colunas 'Codigo Item' e 'Est.Disp'
itens_codigo_especifico = itens_codigo_especifico[['Codigo Item', 'Est.Disp']]

# Exibindo os itens filtrados
print(itens_codigo_especifico)

       Codigo Item  Est.Disp
19575   4111160404         0
19716   4111160404         0
19817   4111160404         0
19950   4111160404         0


In [27]:
# Consolidando os dados no espd068 para cada combinação de Pedido e Cod Item
espd068_consolidado = espd068.groupby(['Pedido', 'Cod Item'], as_index=False)['Qtde Faturada'].sum()

# Realizando o merge entre espd028 e espd068_consolidado
espd028 = espd028.merge(
    espd068_consolidado,                      # Usando o DataFrame consolidado
    left_on=['Nr.Pedido', 'Codigo Item'],      # Colunas em espd028 para a junção
    right_on=['Pedido', 'Cod Item'],           # Colunas em espd068_consolidado para a junção
    how='left',                                # Mantém todos os registros de espd028
    suffixes=('', '_espd068')                  # Adiciona sufixos para evitar duplicação
)

# Renomeia a coluna 'Qtde Faturada_espd068' para 'Qtde Faturada'
espd028 = espd028.rename(columns={'Qtde Faturada_espd068': 'Qtde Faturada'})

# Removendo colunas duplicadas após o merge
espd028 = espd028.drop(columns=['Pedido', 'Cod Item'])

# Preenchendo valores ausentes com 0
espd028['Qtde Faturada'] = espd028['Qtde Faturada'].fillna(0)
espd028['Qtde Faturada'] = espd028['Qtde Faturada'].astype(int)

In [28]:
espd028.head(50)

,Data Implant,Nr.Pedido,Nr.Cliente,Nome Abrev,Nome Repre,Familia,Codigo Item,Descricao,Qt.Pedido,R$ Unit,Qt.Aloc,Vlr.Total,Est.Disp,Situação,Entrega,MêsAno,Lista Presente,Observação,Qtde Faturada
0,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111110369,Flor de Ameixeira,3,"149,6656",0,"448,9968",17,Aberto,15/05/25,52025,0,NaN,0
1,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111110698,CAIXA GRANDE DE PEÇAS CRIATIVAS LEGO,1,"319,6635",0,"319,6635",0,Aberto,15/05/25,52025,0,NaN,0
2,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111110788,Casa Mágica da Gabby,1,"406,2435",0,"406,2435",6,Aberto,15/05/25,52025,0,NaN,0
3,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111111023,Base de ConstruçãoVerde,24,"42,6172",0,"1022,8128",0,Aberto,15/05/25,52025,0,NaN,0
4,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111111025,Base de ConstruçãoAzul,24,"42,6172",0,"1022,8128",21,Aberto,15/05/25,52025,0,NaN,0
5,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111111039,Amigos Criativos da Comida,4,"48,1065",0,"192,426",0,Aberto,15/05/25,52025,0,NaN,0
6,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111111043,Caixa Criativa Legal,2,"149,6625",0,"299,325",11,Aberto,15/05/25,52025,0,NaN,0
7,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111121178,Pousada da Raposa,9,"106,548",0,"958,932",144,Aberto,15/05/25,52025,0,NaN,0
8,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111121251,Expedição do Steveao Deserto,5,"53,4495",0,"267,2475",259,Aberto,15/05/25,52025,0,NaN,0
9,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111121255,A Emboscada no Portal do Nether,3,"187,083",0,"561,249",0,Aberto,15/05/25,52025,0,NaN,0


In [29]:
qtde_faturada_maior_que_zero = espd028[espd028['Qtde Faturada'] > 0]
qtde_faturada_maior_que_zero.head(100)

,Data Implant,Nr.Pedido,Nr.Cliente,Nome Abrev,Nome Repre,Familia,Codigo Item,Descricao,Qt.Pedido,R$ Unit,Qt.Aloc,Vlr.Total,Est.Disp,Situação,Entrega,MêsAno,Lista Presente,Observação,Qtde Faturada
1001,2025-05-09,3464519,1941374,NABIH FARES,JEAN KARLO,TOYSLEGO,4111110432,Viagem de Barco daPeppa pig,0,"164,47",2,"328,94",0,Parcial,09/05/25,52025,0,ok,2
1002,2025-05-09,3464519,1941374,NABIH FARES,JEAN KARLO,TOYSLEGO,4111110452,Viagem de Acampamento,0,"109,64",2,"219,28",1,Parcial,09/05/25,52025,0,ok,2
1004,2025-05-09,3464519,1941374,NABIH FARES,JEAN KARLO,TOYSLEGO,4111110795,Criando com Bebê Caixa,0,"82,23",5,"411,15",19,Parcial,09/05/25,52025,0,ok,5
1010,2025-05-09,3464519,1941374,NABIH FARES,JEAN KARLO,TOYSLEGO,4111160474,F1 Grid com carrosde corrida VCARB,0,"153,5",6,921,40,Parcial,09/05/25,52025,0,ok,6
1011,2025-05-09,3464519,1941374,NABIH FARES,JEAN KARLO,TOYSLEGO,4111171048,Serie 27,0,"27,41",36,"986,76",0,Parcial,09/05/25,52025,0,ok,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,2025-05-08,3463320,9571,CRIMAR01,Marcio,TOYSLEGO,4111176276,Armadura Mech Venom vs. Miles Morale,0,"90,9354",4,"363,74",136,Parcial,08/05/25,52025,0,ok,4
1379,2025-05-08,3463320,9571,CRIMAR01,Marcio,TOYSLEGO,4111176920,Carro Esportivo Ford Mustang Dark Ho,0,"151,5621",3,"454,69",46,Parcial,08/05/25,52025,0,ok,3
1380,2025-05-08,3463320,9571,CRIMAR01,Marcio,TOYSLEGO,4111176925,Aston Martin Vantage F1 Safety Car e,0,"260,6883",2,"521,38",4,Parcial,08/05/25,52025,0,ok,2
1382,2025-05-08,3463320,9571,CRIMAR01,Marcio,TOYSLEGO,4111177251,Carro de corrida McLaren F1® Team MC,0,"151,5621",3,"454,69",42,Parcial,08/05/25,52025,0,ok,3


In [30]:
# Atualizando a coluna 'Qt.Aloc' subtraindo o valor da coluna 'Qtde Faturada'
espd028['Qt.Aloc'] = espd028['Qt.Aloc'] - espd028['Qtde Faturada']

# Verificando as primeiras linhas para garantir que a operação foi realizada corretamente
espd028.head()



,Data Implant,Nr.Pedido,Nr.Cliente,Nome Abrev,Nome Repre,Familia,Codigo Item,Descricao,Qt.Pedido,R$ Unit,Qt.Aloc,Vlr.Total,Est.Disp,Situação,Entrega,MêsAno,Lista Presente,Observação,Qtde Faturada
0,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111110369,Flor de Ameixeira,3,"149,6656",0,"448,9968",17,Aberto,15/05/25,52025,0,NaN,0
1,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111110698,CAIXA GRANDE DE PEÇAS CRIATIVAS LEGO,1,"319,6635",0,"319,6635",0,Aberto,15/05/25,52025,0,NaN,0
2,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111110788,Casa Mágica da Gabby,1,"406,2435",0,"406,2435",6,Aberto,15/05/25,52025,0,NaN,0
3,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111111023,Base de ConstruçãoVerde,24,"42,6172",0,"1022,8128",0,Aberto,15/05/25,52025,0,NaN,0
4,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111111025,Base de ConstruçãoAzul,24,"42,6172",0,"1022,8128",21,Aberto,15/05/25,52025,0,NaN,0


In [31]:
qtde_faturada_maior_que_zero = espd028[espd028['Qtde Faturada'] > 0]
qtde_faturada_maior_que_zero.head(100)

,Data Implant,Nr.Pedido,Nr.Cliente,Nome Abrev,Nome Repre,Familia,Codigo Item,Descricao,Qt.Pedido,R$ Unit,Qt.Aloc,Vlr.Total,Est.Disp,Situação,Entrega,MêsAno,Lista Presente,Observação,Qtde Faturada
1001,2025-05-09,3464519,1941374,NABIH FARES,JEAN KARLO,TOYSLEGO,4111110432,Viagem de Barco daPeppa pig,0,"164,47",0,"328,94",0,Parcial,09/05/25,52025,0,ok,2
1002,2025-05-09,3464519,1941374,NABIH FARES,JEAN KARLO,TOYSLEGO,4111110452,Viagem de Acampamento,0,"109,64",0,"219,28",1,Parcial,09/05/25,52025,0,ok,2
1004,2025-05-09,3464519,1941374,NABIH FARES,JEAN KARLO,TOYSLEGO,4111110795,Criando com Bebê Caixa,0,"82,23",0,"411,15",19,Parcial,09/05/25,52025,0,ok,5
1010,2025-05-09,3464519,1941374,NABIH FARES,JEAN KARLO,TOYSLEGO,4111160474,F1 Grid com carrosde corrida VCARB,0,"153,5",0,921,40,Parcial,09/05/25,52025,0,ok,6
1011,2025-05-09,3464519,1941374,NABIH FARES,JEAN KARLO,TOYSLEGO,4111171048,Serie 27,0,"27,41",0,"986,76",0,Parcial,09/05/25,52025,0,ok,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,2025-05-08,3463320,9571,CRIMAR01,Marcio,TOYSLEGO,4111176276,Armadura Mech Venom vs. Miles Morale,0,"90,9354",0,"363,74",136,Parcial,08/05/25,52025,0,ok,4
1379,2025-05-08,3463320,9571,CRIMAR01,Marcio,TOYSLEGO,4111176920,Carro Esportivo Ford Mustang Dark Ho,0,"151,5621",0,"454,69",46,Parcial,08/05/25,52025,0,ok,3
1380,2025-05-08,3463320,9571,CRIMAR01,Marcio,TOYSLEGO,4111176925,Aston Martin Vantage F1 Safety Car e,0,"260,6883",0,"521,38",4,Parcial,08/05/25,52025,0,ok,2
1382,2025-05-08,3463320,9571,CRIMAR01,Marcio,TOYSLEGO,4111177251,Carro de corrida McLaren F1® Team MC,0,"151,5621",0,"454,69",42,Parcial,08/05/25,52025,0,ok,3


In [32]:
espd028['Qt.Pedido'] = np.where(
    espd028['Situação'] == 'Total', 
    0, 
    np.where(
        espd028['Qt.Pedido'].astype(float) - espd028['Qt.Aloc'] < 0, 
        0, 
        espd028['Qt.Pedido'].astype(float) - espd028['Qt.Aloc']
    )
)

# Convertendo a coluna 'Qt.Pedido' para o tipo int
espd028['Qt.Pedido'] = espd028['Qt.Pedido'].astype(int) 

In [33]:
espd028.head()

,Data Implant,Nr.Pedido,Nr.Cliente,Nome Abrev,Nome Repre,Familia,Codigo Item,Descricao,Qt.Pedido,R$ Unit,Qt.Aloc,Vlr.Total,Est.Disp,Situação,Entrega,MêsAno,Lista Presente,Observação,Qtde Faturada
0,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111110369,Flor de Ameixeira,3,"149,6656",0,"448,9968",17,Aberto,15/05/25,52025,0,NaN,0
1,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111110698,CAIXA GRANDE DE PEÇAS CRIATIVAS LEGO,1,"319,6635",0,"319,6635",0,Aberto,15/05/25,52025,0,NaN,0
2,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111110788,Casa Mágica da Gabby,1,"406,2435",0,"406,2435",6,Aberto,15/05/25,52025,0,NaN,0
3,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111111023,Base de ConstruçãoVerde,24,"42,6172",0,"1022,8128",0,Aberto,15/05/25,52025,0,NaN,0
4,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111111025,Base de ConstruçãoAzul,24,"42,6172",0,"1022,8128",21,Aberto,15/05/25,52025,0,NaN,0


In [34]:
transito = pd.read_excel('transito.xlsx', header=0)

# Identificar colunas com texto no cabeçalho
text_columns = transito.columns[transito.columns.map(lambda x: isinstance(x, str))]

# Exibir os nomes das colunas de texto
print("Colunas com texto no cabeçalho:", text_columns.tolist())

# Fazer o merge do DataFrame espd028 com transito.xlsx usando 'Codigo Item' e 'Cod'
espd028 = espd028.merge(
    transito,
    left_on='Codigo Item',
    right_on='Cod',
    how='left'
)
for column in text_columns:
    espd028[column] = espd028[column].fillna(0).astype(int)
espd028 = espd028.drop(columns=['Cod'])

Colunas com texto no cabeçalho: ['Cod', 'Chegada dia 15/05', 'Disponivel 12/Mai', '22/05\n(estimated)', '27/05\n(estimated)', '30/05\n(estimated)', '03/06\n(estimated)', '13/06\n(estimated)', '18/06\n(estimated)']


In [35]:
espd028.head(500)

,Data Implant,Nr.Pedido,Nr.Cliente,Nome Abrev,Nome Repre,Familia,Codigo Item,Descricao,Qt.Pedido,R$ Unit,...,Observação,Qtde Faturada,Chegada dia 15/05,Disponivel 12/Mai,22/05\n(estimated),27/05\n(estimated),30/05\n(estimated),03/06\n(estimated),13/06\n(estimated),18/06\n(estimated)
0,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111110369,Flor de Ameixeira,3,"149,6656",...,NaN,0,3,1544,540,90,0,360,0,0
1,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111110698,CAIXA GRANDE DE PEÇAS CRIATIVAS LEGO,1,"319,6635",...,NaN,0,106,268,1410,0,0,300,450,0
2,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111110788,Casa Mágica da Gabby,1,"406,2435",...,NaN,0,0,122,0,30,0,0,0,0
3,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111111023,Base de ConstruçãoVerde,24,"42,6172",...,NaN,0,240,252,576,144,0,0,576,0
4,2025-05-15,3470072,1582510,FLIP8,DPS COMERCIA,TOYSLEGO,4111111025,Base de ConstruçãoAzul,24,"42,6172",...,NaN,0,12,1068,0,0,0,144,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2025-05-14,3468978,1649851,CABRAL2,RHOL REPRESE,TOYSLEGO,4111160450,Carregadeira de Rodas de Construcao,4,"77,2962",...,NaN,0,8,1224,0,0,0,0,0,348
496,2025-05-14,3468978,1649851,CABRAL2,RHOL REPRESE,TOYSLEGO,4111160474,F1 Grid com carrosde corrida VCARB,3,"144,29",...,NaN,0,9,1254,0,0,0,0,0,0
497,2025-05-14,3468978,1649851,CABRAL2,RHOL REPRESE,TOYSLEGO,4111176301,Batman e Batmovelvs. Sr. Frio,5,"92,7592",...,NaN,0,0,825,0,0,0,1740,0,580
498,2025-05-14,3468978,1649851,CABRAL2,RHOL REPRESE,TOYSLEGO,4111176302,Robo do Superman vs. Lex Luthor,5,"77,2962",...,NaN,0,25,1399,435,290,0,1885,0,580


In [36]:
text_columns = transito.columns.tolist()
if 'Cod' in text_columns:
    text_columns.remove('Cod')
text_columns.insert(0, 'Est.Disp')

In [37]:
estoque_inicial = espd028[['Codigo Item'] + text_columns].copy()

# Para cada código de item, manter apenas o primeiro valor de estoque
estoque_inicial = estoque_inicial.drop_duplicates(subset='Codigo Item')

# Preencher valores NaN com 0 e converter para int
estoque_inicial[text_columns] = estoque_inicial[text_columns].fillna(0).astype(int)


# Exibir as primeiras linhas do DataFrame
print(estoque_inicial.head(10))



   Codigo Item  Est.Disp  Chegada dia 15/05  Disponivel 12/Mai  \
0   4111110369        17                  3               1544   
1   4111110698         0                106                268   
2   4111110788         6                  0                122   
3   4111111023         0                240                252   
4   4111111025        21                 12               1068   
5   4111111039         0                268                636   
6   4111111043        11                  9               1524   
7   4111121178       144                 60               4692   
8   4111121251       259                 50               1230   
9   4111121255         0                246                396   

   22/05\n(estimated)  27/05\n(estimated)  30/05\n(estimated)  \
0                 540                  90                   0   
1                1410                   0                   0   
2                   0                  30                   0   
3            

In [38]:
import random

espd028['Valor Item'] = espd028['R$ Unit'].apply(lambda x: float(x.replace(',', '.'))) * espd028['Qt.Pedido']

# Agrupa por pedido e calcula o total
pedidos_totais = espd028.groupby('Nr.Pedido', as_index=False)['Valor Item'].sum()
pedidos_totais = pedidos_totais.rename(columns={'Valor Item': 'Valor Total Pedido'})

# Junta o total com os dados originais
espd028 = espd028.merge(pedidos_totais, on='Nr.Pedido', how='left')

# 2. Definir as estratégias de ordenação dos pedidos
# Obter a lista única de pedidos
pedidos = espd028['Nr.Pedido'].unique()

# Estratégia 1: Maior para Menor (usando o total do pedido)
pedidos_maior_para_menor = list(pedidos_totais.sort_values(by='Valor Total Pedido', ascending=False)['Nr.Pedido'])

# Estratégia 2: Menor para Maior
pedidos_menor_para_maior = list(pedidos_totais.sort_values(by='Valor Total Pedido', ascending=True)['Nr.Pedido'])

# Estratégia 3 e 4: Ordens aleatórias
pedidos_aleatorio1 = list(pedidos)
pedidos_aleatorio2 = list(pedidos)
random.shuffle(pedidos_aleatorio1)
random.shuffle(pedidos_aleatorio2)

# Organiza as estratégias em um dicionário
estrategias = {
    'Maior para Menor': pedidos_maior_para_menor,
    'Menor para Maior': pedidos_menor_para_maior,
    'Aleatório 1': pedidos_aleatorio1,
    'Aleatório 2': pedidos_aleatorio2
}

# 3. Função para simular o faturamento para uma dada ordem de pedidos
def simular_faturamento(pedidos_ordem, espd028, estoque_inicial, text_columns):
    faturado_total = []
    # Cria uma cópia do estoque para a simulação
    estoque_sim = estoque_inicial.copy(deep=True)
    ordem_faturamento = 0

    # Para cada pedido na ordem definida
    for pedido in pedidos_ordem:
        ordem_faturamento += 1  # Incrementa a ordem de processamento do pedido
        # Seleciona os itens deste pedido (mantendo a ordem original)
        itens_pedido = espd028[espd028['Nr.Pedido'] == pedido].copy()

        # Para cada coluna de estoque, em ordem
        for estoque_col in text_columns:
            # Itera sobre cada item do pedido que ainda possui quantidade pendente
            for idx, item in itens_pedido.iterrows():
                qt_pedido = item['Qt.Pedido']
                if qt_pedido <= 0:
                    continue  # Item já faturado completamente

                cliente = item['Nr.Cliente']
                familia = item['Familia']
                codigo_item = item['Codigo Item']
                valor_unit = float(item['R$ Unit'].replace(',', '.'))

                # Se o item não existe no estoque, pula
                if codigo_item not in estoque_sim['Codigo Item'].values:
                    continue

                # Obter a linha do estoque correspondente
                estoque_item = estoque_sim[estoque_sim['Codigo Item'] == codigo_item].iloc[0]
                estoque_atual = estoque_item[estoque_col]

                if estoque_atual > 0:
                    qt_faturada = min(qt_pedido, estoque_atual)
                    valor_faturado = qt_faturada * valor_unit

                    faturado_total.append({
                        'Nr.Pedido': pedido,
                        'Nr.Cliente': cliente,
                        'Familia': familia,
                        'Codigo Item': codigo_item,
                        'Qtde Faturada': qt_faturada,
                        'Valor Faturado': valor_faturado,
                        'Estoque Faturado': estoque_col,
                        'Ordem Faturamento Pedido': ordem_faturamento,
                    })

                    # Atualiza a quantidade restante do item e o estoque
                    itens_pedido.loc[idx, 'Qt.Pedido'] -= qt_faturada
                    estoque_sim.loc[estoque_sim['Codigo Item'] == codigo_item, estoque_col] -= qt_faturada

        # Se desejar, pode verificar se algum item não foi completamente faturado e registrar isso
    # Retorna o DataFrame com os resultados da simulação
    return pd.DataFrame(faturado_total)
print(estoque_inicial.head())
# 4. Executar a simulação para cada estratégia e salvar as abas em um arquivo Excel
with pd.ExcelWriter('Faturado_Total_Estrategias.xlsx') as writer:
    for nome_estrategia, pedidos_ordem in estrategias.items():
        resultado = simular_faturamento(pedidos_ordem, espd028, estoque_inicial, text_columns)
        resultado.to_excel(writer, sheet_name=nome_estrategia, index=False)

print('Simulação de faturamento por pedido concluída. Arquivo "Faturado_Total_Estrategias.xlsx" gerado.')

   Codigo Item  Est.Disp  Chegada dia 15/05  Disponivel 12/Mai  \
0   4111110369        17                  3               1544   
1   4111110698         0                106                268   
2   4111110788         6                  0                122   
3   4111111023         0                240                252   
4   4111111025        21                 12               1068   

   22/05\n(estimated)  27/05\n(estimated)  30/05\n(estimated)  \
0                 540                  90                   0   
1                1410                   0                   0   
2                   0                  30                   0   
3                 576                 144                   0   
4                   0                   0                   0   

   03/06\n(estimated)  13/06\n(estimated)  18/06\n(estimated)  
0                 360                   0                   0  
1                 300                 450                   0  
2                   

Simulação de faturamento por pedido concluída. Arquivo "Faturado_Total_Estrategias.xlsx" gerado.


In [39]:
faturado_total = []
espd028.to_excel('Teste.xlsx',index=False)
# Iterar sobre cada item em espd028
for idx, item in espd028.iterrows():
    pedido = item['Nr.Pedido']
    cliente = item['Nr.Cliente']
    familia = item['Familia']
    codigo_item = item['Codigo Item']
    qt_pedido = item['Qt.Pedido']
    valor_unit = float(item['R$ Unit'].replace(',', '.'))  # Convertendo o valor para float

    # Verificar se o item está no estoque_inicial
    if codigo_item not in estoque_inicial['Codigo Item'].values:
        continue  # Item não tem estoque registrado
    
    # Obter a linha correspondente ao item
    estoque_item = estoque_inicial[estoque_inicial['Codigo Item'] == codigo_item].iloc[0]
    
    # Iterar sobre as colunas de estoque para faturar
    for estoque_col in text_columns:  # Ignorar 'Est.Disp'
        estoque_atual = estoque_item[estoque_col]
        if qt_pedido > 0 and estoque_atual > 0:
            qt_faturada = min(qt_pedido, estoque_atual)
            valor_faturado = qt_faturada * valor_unit
            
            # Adicionar a linha de faturamento ao resultado
            faturado_total.append({
                'Nr.Pedido': pedido,
                'Nr.Cliente': cliente,
                'Familia': familia,
                'Codigo Item': codigo_item,
                'Qtde Faturada': qt_faturada,
                'Valor Faturado': valor_faturado,
                'Estoque Faturado': estoque_col
            })
            
            # Atualizar a quantidade a ser faturada e o estoque atual
            qt_pedido -= qt_faturada
            estoque_inicial.loc[estoque_inicial['Codigo Item'] == codigo_item, estoque_col] -= qt_faturada
            
            if qt_pedido <= 0:
                break
       

#Criar o DataFrame final
faturado_total_df = pd.DataFrame(faturado_total)

# Salvar o DataFrame em um arquivo Excel, se necessário
faturado_total_df.to_excel('Faturado_Total.xlsx', index=False)



print('fim')

fim
